# Baseline Accuracy Table

MODEL × DATASET accuracy matrix

In [ ]:
import pandas as pd
from data_reader.json_reader import get_baseline_stats
from constants import ALL_MODELS, DATASETS

In [2]:
results = []
row_counts = []

for model in ALL_MODELS:
    row = {"Model": model}
    counts = {"Model": model}
    
    for dataset in DATASETS:
        # med_qa: split=None combines all splits automatically
        # others: split=None defaults to "test"
        stats = get_baseline_stats(dataset_name=dataset, model=model, split=None)
        
        if stats:
            row[dataset] = stats["accuracy"]
            counts[dataset] = stats["n_total"]
        else:
            row[dataset] = None
            counts[dataset] = 0
    
    results.append(row)
    row_counts.append(counts)
    print(f"Done: {model}")

baseline_df = pd.DataFrame(results)
counts_df = pd.DataFrame(row_counts)

Done: deepseek-ai_DeepSeek-R1-Distill-Qwen-32B
Done: openai_gpt-oss-120b


In [4]:
# Format and display
display_df = baseline_df.copy()
display_df = display_df.set_index("Model")

# Format as percentages
styled = display_df.style.format("{:.1%}", na_rep="-").background_gradient(cmap="RdYlGn", axis=None)
styled

,med_qa,AfrimedQA,MedBullets,MedExQA,MedMCQA,MedXpertQA-R,MedXpertQA-U,MMLU-Pro,MMLU,PubMedQA
Model,,,,,,,,,,
deepseek-ai_DeepSeek-R1-Distill-Qwen-32B,77.8%,71.3%,59.1%,81.0%,67.2%,18.5%,21.2%,67.0%,90.3%,75.0%
openai_gpt-oss-120b,92.1%,76.4%,83.8%,84.6%,76.1%,37.0%,34.3%,74.9%,92.4%,73.8%
